In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
twitch = pd.read_csv('project_data.csv')
twitch.head(5)

,time,day_of_week,city,country,channel,status,game,avg_ccu,preliminary_tag
0,6/9/2016 3:48,4,Boardman,US,dotamajor,Fnatic vs Newbee - The Manila Major - Main Event,Dota 2,121426,non_community
1,8/20/2016 8:17,6,Boardman,US,esl_overwatch,REUNITED vs. Rogue - Overwatch Atlantic Showdo...,Overwatch,51202,non_community
2,6/10/2016 18:11,5,Boardman,US,dotamajor,MVP vs LGD Gaming - The Manila Major - Main Event,Dota 2,39902,non_community
3,8/15/2016 1:05,1,Boardman,US,nl_kripp,Back@9pm EDT / 3am CEST! (✿☯‿☯✿)(✿☯‿☯✿),Hearthstone: Heroes of Warcraft,28930,community
4,6/16/2016 14:08,4,Boardman,US,starladder_hs_en,Surrender vs Lifecoach | SL i-League StarSerie...,Hearthstone: Heroes of Warcraft,21960,non_community


In [3]:
twitch.shape

(1000, 9)

In [4]:
game_dummies = pd.get_dummies(twitch['game'], prefix='game_').iloc[:, 1:]

# concatenate the dummy variable columns onto the original DataFrame (axis=0 means rows, axis=1 means columns)
twitch = pd.concat([twitch, game_dummies], axis=1)
twitch.head(3)
twitch.shape

# this adds an additional 444 columns (for the 444 different games)

channel_dummies = pd.get_dummies(twitch['channel'], prefix='channel_').iloc[:, 1:]

twitch = pd.concat([twitch, channel_dummies], axis = 1)
twitch.shape

# this adds an additional 1178 columns 

(1000, 743)

In [ ]:
class TwitchTransformer(TransformerMixin):
    def transform(self, X, **transform_params): 
        new_X = pd.get_dummies(X['game'], prefix='game_').iloc[:, 1:]
        new_X.concat(numerial_columns)

        # concatenate the dummy variable columns onto the original DataFrame (axis=0 means rows, axis=1 means columns)
        channel_dummies = pd.get_dummies(X['channel'], prefix='channel_').iloc[:, 1:]
        new_X = pd.concat([new_X, channel_dummies], axis = 1)
        return new_X

        # this adds an additional 1178 columns 

    def fit(self, X, y=None, **fit_params):
        return self

In [5]:
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from textblob import TextBlob, Word

In [6]:
vect = CountVectorizer(stop_words='english', max_features = 250)
titles = pd.DataFrame(vect.fit_transform(twitch['status']).toarray(), columns=vect.get_feature_names())
titles

,000,10,100,1000,11,12,12pm,14,1nd,1st,...,wins,wn8,world,wow,xb1,xcomchanges,xp,youtube,ziqoftw,zombies
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
